# Simurgh


1. Launch simulator
2. Launch `bluebird`
3. Query aircraft positions
4. Load sector into simulator using `bluebird`
5. Load scenario into simulator using `bluebird`
6. Query aircraft positions
7. Explain OpenAI interface
8. Explain simple step
9. Run for loop of 10 steps with rule based approach.


### Launch simulator and `bluebird` interface layer

In [1]:
# Machine College
!docker-compose --file ../bluebird/tests/integration/machcoll/docker-compose.yml up --detach
# Bluesky
# !docker-compose --file ../docker-compose.yml up --detach

Creating network "machcoll_default" with the default driver
Creating rabbitmq_integration ... 
ting rabbitmq_integration ... doneCreating machcoll_metrics_integration ... 
Creating machcoll_integration         ... mdone
ting machcoll_integration         ... done

Import `pydodo`

In [2]:
import pydodo

In [3]:
# print(pydodo.bluebird_connect.get_bluebird_url())
# pydodo.bluebird_config(port="5321")

In [4]:
pydodo.bluebird_config

<function pydodo.bluebird_connect.bluebird_config(host='localhost', port='5001', version='v2')>

### Query aircraft positions

Assuming one has already set up a Conda environment and installed the dependencies via: 
```$ source install.sh```

Then importing `pydodo` should allow for a quick query of aircraft posistions (there should be none as we have not uploaded any sector or scenario information yet)

In [5]:
import pydodo
pydodo.all_positions()

,aircraft_type,ground_speed,latitude,longitude,vertical_speed,heading,current_flight_level,requested_flight_level,cleared_flight_level


_Success!_

What do you mean success?! What we are seeing here is that `pydodo` is able to query positions of planes inside the simulator. The fact we do not see any is only becuase we have not defined a "Sector" and a "Scenario" yet.

## What is a Sector?

## What is a Scenario?

In [6]:
pydodo.simulation_info()

AttributeError: module 'pydodo' has no attribute 'simulation_info'

Remeber a Sector upload comes _before_ a Scenario. Why? Because it would not make sense to define a set of aircraft in a area that will not be observed.

### Load sector into `bluesky` using `bluebird`

In [7]:
pydodo.upload_sector('../dodo/PyDodo/tests/dodo-test-sector.geojson', 'sector')

True

### Load scenario into `bluesky` using `bluebird`

In [8]:
pydodo.upload_scenario('../dodo/PyDodo/tests/dodo-test-scenario.json', 'scenario')

True

In [9]:
pydodo.simulation_info()

AttributeError: module 'pydodo' has no attribute 'simulation_info'

### Query aircraft positions, again

In [12]:
pydodo.all_positions()

,aircraft_type,ground_speed,latitude,longitude,vertical_speed,heading,current_flight_level,requested_flight_level,cleared_flight_level


In [13]:
pydodo.list_route('VJ159')

HTTPError: Aircraft "VJ159" does not exist

In [14]:
pydodo.simulation_step()

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:5001/api/v2/step

In [15]:
pydodo.all_positions()

,aircraft_type,ground_speed,latitude,longitude,vertical_speed,heading,current_flight_level,requested_flight_level,cleared_flight_level


### Explain OpenAI interface

### Explain simple step

### Run for loop of 10 steps with rule based approach.

This example follows a simple rule based set of steps which check to see if a group of planes are away from their requested flight levels. Then for the plane that is furthest away from it's requested flight level, it send the command to that plane to _go_ to its requested flight level.

In [ ]:
# %load ../dodo/PyDodo/plan.py
import gym
import birdhouse

env = gym.make("birdhouse-v1")

action = None

for _ in range(10):
    # ENVIRONMENT STEP
    obs, reward, done, info = env.step(action)
    # AGENT - ACTION SELECTION
    # --> choose plan wit biggest absolute difference between current and requested flight levels
    # and send to requested FL (unless the difference is < 10 FL)
    obs["FL_diff"] = (obs['requested_flight_level'] - obs['current_flight_level']/100).abs()
    max_diff =  obs.loc[obs['FL_diff'].idxmax()]
    if max_diff['FL_diff'] >= 10:
        callsign = max_diff.name
        req_fl = max_diff['requested_flight_level']
        action = (callsign, req_fl)
    else:
        action = None